In [140]:
%%writefile projections.py

import urllib3
from bs4 import BeautifulSoup
from collections import defaultdict

def convert_string(s):
    try:
        return float(s)
    except:
        return 0.0
    
def get_nf_projections():
    base_page = 'https://www.numberfire.com/nfl/fantasy/fantasy-football-projections'
    http = urllib3.PoolManager()
    response = http.request('GET', base_page)
    soup = BeautifulSoup(response.data, 'html.parser')
    rows = soup.find_all('tr')
    plyr_list = []
    projections = []
    for row in rows:
        if row.a:
            plyr_list.append(row.a.get_text())
        if len(row)==47:
            score = [convert_string(td.string) for td in row.find_all('td', {'class': 'nf_fp active'})]
            if len(score)>0:
                projections.append(score[0])
    plyr_list = [p.split('\n')[1] for p in plyr_list]
    projection_dict = {x: y for (x,y) in zip(plyr_list, projections)}
    return projection_dict

def get_espn_projections():
    base_page = 'http://games.espn.com/ffl/tools/projections?'
    addon = 'startIndex='
    startindex = list(range(40, 1080, 40))
    plyr_dict = defaultdict(dict)
    page = base_page
    for i in startindex:
        http = urllib3.PoolManager()
        response = http.request('GET', page)
        soup = BeautifulSoup(response.data, 'html.parser')
        rows = soup.find_all('tr')
        for row in rows:
            if len(row) == 14:
                if row.a.get_text() != 'PLAYER':
                    plyr_dict[row.a.get_text()] = [convert_string(td.string) for td in \
                                                   row.find_all('td', \
                                                                {'class': 'playertableStat appliedPoints sortedCell'})][0]
        page = base_page + addon + str(i)

    d_plyr_dict = {x.split(' ')[0]: y for (x,y) in plyr_dict.items() if x.split(' ')[1] == 'D/ST'}
    return plyr_dict, d_plyr_dict

Overwriting projections.py


In [142]:
get_nf_projections()

{'Kyler Murray': 23.19,
 'Josh Allen': 22.89,
 'Russell Wilson': 22.04,
 'Lamar Jackson': 20.37,
 'Deshaun Watson': 19.94,
 'Justin Herbert': 19.24,
 'Drew Brees': 19.16,
 'Aaron Rodgers': 19.02,
 'Jared Goff': 18.77,
 'Dalvin Cook': 18.44,
 'Matthew Stafford': 18.05,
 'Aaron Jones': 17.97,
 'Tom Brady': 17.69,
 'Derek Carr': 17.68,
 'Carson Wentz': 17.62,
 'Drew Lock': 17.34,
 'Teddy Bridgewater': 17.3,
 'Ryan Tannehill': 17.29,
 'Derrick Henry': 17.12,
 'Tua Tagovailoa': 16.94,
 'Ben Roethlisberger': 16.82,
 'Alvin Kamara': 16.51,
 'Alex Smith': 16.5,
 'Joe Burrow': 16.21,
 'Cam Newton': 16.13,
 'Philip Rivers': 15.78,
 'Baker Mayfield': 15.63,
 'Nick Mullens': 15.62,
 'Daniel Jones': 15.35,
 'Josh Jacobs': 14.45,
 'Nick Foles': 14.4,
 'Davante Adams': 14.16,
 'Miles Sanders': 13.9,
 'Kirk Cousins': 13.9,
 'James Conner': 13.66,
 'Nick Chubb': 13.54,
 'James Robinson': 13.49,
 'Jake Luton': 13.26,
 'DeAndre Hopkins': 13.25,
 'Terry McLaurin': 12.49,
 'Michael Thomas': 12.3,
 'D.K. Me